In [1]:
def fff(x): # converting .txt to list
  with open(x) as file: 
    line = []
    for lines in file.readlines():
      line.append(lines)
    return line
line = fff('train.txt')

In [2]:
line[0:5]

['i didnt feel humiliated;sadness\n',
 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake;sadness\n',
 'im grabbing a minute to post i feel greedy wrong;anger\n',
 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property;love\n',
 'i am feeling grouchy;anger\n']

In [3]:
import pandas as pd
import re

In [4]:
def csv(line): # converting list into data frame
  list1,list2 = [],[]
  for lines in line:
    x,y = lines.split(';')
    y = y.replace('\n','')
    list1.append(x)
    list2.append(y)
  df = pd.DataFrame(list(list1),columns=['sentence'])
  df['emotion'] = list2
  return df

In [5]:
df = csv(line)
# df
# df.describe
# df.shape
# df['emotion'].unique()
# df.emotion.value_counts()
# df.isnull().sum()

In [6]:
df.emotion.value_counts()

joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: emotion, dtype: int64

In [8]:
import nltk
# nltk.download('wordnet')
# nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

In [9]:
wn = WordNetLemmatizer()

In [10]:
'''
def lem(x): # lemmating the text
  corpus = []
  i=1
  for words in x:
    words = words.split()
    y = [wn.lemmatize(word) for word in words if not word in stopwords.words('english')]
    y =  ' '.join(y)
    corpus.append(y)
  return corpus
x = lem(df['sentence']) # list
'''

In [11]:
'''
# saving the lemmatized list as text
f = open("train_x_lem.txt", "w")
for item in x:
   f.write(item + "\n")
f.close()
'''

In [20]:
# extracting the saved lemmatized text as a list
# xx = fff('train_x_lem.txt')
x = []
with open('train_x_lem.txt') as x:
    x = [line.strip() for line in x]
x

['didnt feel humiliated',
 'go feeling hopeless damned hopeful around someone care awake',
 'im grabbing minute post feel greedy wrong',
 'ever feeling nostalgic fireplace know still property',
 'feeling grouchy',
 'ive feeling little burdened lately wasnt sure',
 'ive taking milligram time recommended amount ive fallen asleep lot faster also feel like funny',
 'feel confused life teenager jaded year old man',
 'petronas year feel petronas performed well made huge profit',
 'feel romantic',
 'feel like make suffering seeing mean something',
 'feel running divine experience expect type spiritual encounter',
 'think easiest time year feel dissatisfied',
 'feel low energy thirsty',
 'immense sympathy general point possible proto writer trying find time write corner life sign agent let alone publishing contract feel little precious',
 'feel reassured anxiety side',
 'didnt really feel embarrassed',
 'feel pretty pathetic time',
 'started feeling sentimental doll child began collection vint

### Handling Text Data

In [14]:
test_line = fff('test.txt')

In [15]:
test_df = csv(test_line)
# test_df.head()

In [19]:
'''
x_test = lem(test_df['sentence']) # lemmatizing
# saving the lemmatized list as text
f = open("test_x_lem.txt", "w")
for item in x_test:
   f.write(item + "\n")
f.close()
'''

In [22]:
# '''
x_test = []
with open('test_x_lem.txt') as xx:
    x_test = [line.strip() for line in xx]
x_test
# '''

['im feeling rather rotten im ambitious right',
 'im updating blog feel shitty',
 'never make separate ever want feel like ashamed',
 'left bouquet red yellow tulip arm feeling slightly optimistic arrived',
 'feeling little vain one',
 'cant walk shop anywhere feel uncomfortable',
 'felt anger end telephone call',
 'explain clung relationship boy many way immature uncommitted despite excitement feeling getting accepted master program university virginia',
 'like breathless feeling reader eager see happen next',
 'jest feel grumpy tired pre menstrual probably week im fit walrus vacation summer',
 'feel particularly agitated',
 'feel beautifully emotional knowing woman knew handful holding baba journey',
 'pay attention deepens feeling invaded helpless',
 'feel extremely comfortable group people dont even need hide',
 'find odd position feeling supportive',
 'feeling heartbroken im sure katniss',
 'feel little mellow today',
 'feel like role would tear sail pessimism discontent',
 'feel 

In [23]:
test_df.shape

(2000, 2)

In [24]:
all = x + x_test # list
len(all)

18000

In [25]:
y = df.iloc[:,1].values # training data as array
# y
# y.shape

In [26]:
y_test = test_df.iloc[:,1].values  # testing data ar array
y_test.shape # array

(2000,)

## Building and training the lstm model

In [27]:
from tensorflow.keras.layers import Embedding,LSTM,Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

In [28]:
y_train = pd.DataFrame(y) # data frame

In [29]:
# creating a internal dictionary where we are indexing each word
tokenizer = Tokenizer(num_words=10000, split=' ')
tokenizer.fit_on_texts(all)

def conv(all):
    # replacing each word with it's corresponding value acc. to dict
    X1 = tokenizer.texts_to_sequences(all)
    X1 = pad_sequences(X1, maxlen=20, padding='post', truncating='post')
    return X1

# all is list type
X1 = conv(all)
X_train = X1[:16000]
X_test = X1[16000:]


Y_train = pd.get_dummies(y_train).values # creating one hot vector for all unique values of y
Y_test = pd.get_dummies(y_test).values


In [30]:
print(y_test)

['sadness' 'sadness' 'sadness' ... 'joy' 'joy' 'fear']


In [31]:
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(16000, 20) (16000, 6) (2000, 20) (2000, 6)


In [32]:
con = pd.Series(list('acdba'))
print(pd.get_dummies(con))

   a  b  c  d
0  1  0  0  0
1  0  0  1  0
2  0  0  0  1
3  0  1  0  0
4  1  0  0  0


In [33]:
model = Sequential()
model.add(Embedding(input_dim=10000,output_dim = 64,input_length=20)) # dim = dimension, number of dimensions of vector 
model.add(LSTM(64)) # 64 is number of neurons in the LSTM
model.add(Dense(6,activation='softmax'))

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 64)            640000    
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 6)                 390       
                                                                 
Total params: 673,414
Trainable params: 673,414
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.compile(optimizer='rmsprop',loss='mse',metrics=['accuracy'])

In [36]:
model.fit(X_train,Y_train,batch_size=32,epochs=10,verbose=2,validation_split=0.2)

Epoch 1/10
400/400 - 4s - loss: 0.1102 - accuracy: 0.4340 - val_loss: 0.0877 - val_accuracy: 0.6000 - 4s/epoch - 11ms/step
Epoch 2/10
400/400 - 3s - loss: 0.0717 - accuracy: 0.6874 - val_loss: 0.0547 - val_accuracy: 0.7691 - 3s/epoch - 7ms/step
Epoch 3/10
400/400 - 3s - loss: 0.0456 - accuracy: 0.8137 - val_loss: 0.0447 - val_accuracy: 0.8194 - 3s/epoch - 7ms/step
Epoch 4/10
400/400 - 3s - loss: 0.0318 - accuracy: 0.8780 - val_loss: 0.0380 - val_accuracy: 0.8528 - 3s/epoch - 7ms/step
Epoch 5/10
400/400 - 3s - loss: 0.0252 - accuracy: 0.9049 - val_loss: 0.0388 - val_accuracy: 0.8484 - 3s/epoch - 7ms/step
Epoch 6/10
400/400 - 3s - loss: 0.0214 - accuracy: 0.9160 - val_loss: 0.0317 - val_accuracy: 0.8759 - 3s/epoch - 7ms/step
Epoch 7/10
400/400 - 3s - loss: 0.0176 - accuracy: 0.9290 - val_loss: 0.0300 - val_accuracy: 0.8809 - 3s/epoch - 7ms/step
Epoch 8/10
400/400 - 3s - loss: 0.0146 - accuracy: 0.9420 - val_loss: 0.0270 - val_accuracy: 0.8903 - 3s/epoch - 7ms/step
Epoch 9/10
400/400 - 3s

In [37]:
loss,acc = model.evaluate(X_test,Y_test)

63/63 [==============================] - 0s 3ms/step - loss: 0.0289 - accuracy: 0.8905


In [38]:
loss

0.028914140537381172

In [39]:
acc

0.890500009059906

In [40]:
preds = model.predict(X_test)
x_test

['im feeling rather rotten im ambitious right',
 'im updating blog feel shitty',
 'never make separate ever want feel like ashamed',
 'left bouquet red yellow tulip arm feeling slightly optimistic arrived',
 'feeling little vain one',
 'cant walk shop anywhere feel uncomfortable',
 'felt anger end telephone call',
 'explain clung relationship boy many way immature uncommitted despite excitement feeling getting accepted master program university virginia',
 'like breathless feeling reader eager see happen next',
 'jest feel grumpy tired pre menstrual probably week im fit walrus vacation summer',
 'feel particularly agitated',
 'feel beautifully emotional knowing woman knew handful holding baba journey',
 'pay attention deepens feeling invaded helpless',
 'feel extremely comfortable group people dont even need hide',
 'find odd position feeling supportive',
 'feeling heartbroken im sure katniss',
 'feel little mellow today',
 'feel like role would tear sail pessimism discontent',
 'feel 

In [50]:
# checking the accuracy of tesing data
cnt = 0
for i in range(len(preds)):
    idx = 0
    mx = 0
    for j in range(6):
        if(preds[i][j] > mx):
            mx = preds[i][j]
            idx = j
    if(Y_test[i][idx]):
        cnt += 1
print(cnt/len(preds))

0.8905


In [44]:
Y_test.shape

(2000, 6)

In [41]:
preds[96] # how do we know which number indicates what?

array([0.00162857, 0.00103248, 0.7985652 , 0.18336375, 0.00260921,
       0.01280079], dtype=float32)

In [67]:
test_df.shape
test_df[test_df['emotion'] == "love"]

,sentence,emotion
14,i find myself in the odd position of feeling s...,love
71,i feel like a naughty school girl because i am...,love
74,i were to go overseas or cross the border then...,love
79,i want each of you to feel my gentle embrace,love
96,i love neglecting this blog but sometimes i fe...,love
...,...,...
1961,i remember wanting to fit in so bad and feelin...,love
1969,i can feel the warmth of the gentle sun,love
1970,i guess as long as the table in the above is p...,love
1990,i really feel like they were gentle reminders ...,love


In [78]:
# preds2 = model.predict(conv(["I'm in love with you"]))
preds2 = model.predict(X_train)

In [93]:
for x in [6, 32, 57, 64, 129, 15871, 15943, 15970, 15974, 15989]:
    print(preds2[x])

[0.00385912 0.04219286 0.004588   0.01854622 0.00340468 0.9274092 ]
[8.0275256e-04 1.3446640e-02 1.3284393e-03 7.8941332e-03 9.5234631e-04
 9.7557563e-01]
[9.66416497e-04 1.28930155e-02 1.76331261e-03 8.40876438e-03
 1.69221917e-03 9.74276364e-01]
[0.04379647 0.36831084 0.2066968  0.08788773 0.03264157 0.2606665 ]
[0.00655809 0.13243556 0.01609828 0.00851233 0.01275618 0.82363963]
[0.00110526 0.01257042 0.00251686 0.01914457 0.00147402 0.9631888 ]
[0.00109816 0.01600169 0.00198279 0.00737231 0.00202868 0.9715163 ]
[0.01807599 0.4104969  0.05614258 0.02140561 0.02138549 0.47249344]
[0.07278612 0.03864811 0.17821535 0.58085227 0.06742804 0.06207009]
[0.10946368 0.27124932 0.15498014 0.19546963 0.08932719 0.17951001]


In [92]:
y_train[y_train[0] == 'surprise']

,0
6,surprise
32,surprise
57,surprise
64,surprise
129,surprise
...,...
15871,surprise
15943,surprise
15970,surprise
15974,surprise
